In [1]:
import pandas as pd
import numpy as np
import configparser
import pymysql
import requests
import re
from tqdm import tqdm
import configparser
import openai
import os
import json

In [2]:
config=configparser.ConfigParser()
config.read('/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini')

['/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini']

In [276]:
mydb= pymysql.connect(
host=config['iimjobs_db']['hostname'],
user=config['iimjobs_db']['username'],
password=config['iimjobs_db']['password'],
db=config['iimjobs_db']['db_name']
)

mycursor = mydb.cursor()

In [277]:
# mycursor = mydb.cursor()

 
query = "SELECT userid from usr_applied_jobs_info where jobid=1372978"

 # 1372184

mycursor.execute(query)

result = mycursor.fetchall()


In [278]:
udf = pd.DataFrame(result,columns=['userid'])

In [279]:
uid_list = udf.userid.values

In [280]:
uid_list

array([1838334, 1422653, 2152366, 2786265, 1512430,  397888, 2033637,
       2514345,  220243, 1553541, 2121520, 2049964, 1102604, 1540164,
       1643106,  181465, 2789539,  370663, 2237092, 1258769, 2785697,
        869704,  230397,  623738, 2625419, 2765048, 2792035, 2318838,
       2102824, 2789761, 1648670, 2781463, 2776836, 2781338,  659772,
       2772792, 2787606, 2332447, 1668092, 1638081, 2791153, 1606494,
       2784873, 2100995, 1834146, 1042241,  910736, 2795573, 2175992,
       2716065, 1450495, 1276981, 2765970,  661121, 2572410, 1077888,
       2014923,  963142, 1568407,  633376, 2445179, 1755149, 2743880,
        294180,  869598, 2709198, 2337894, 2351233, 2507778, 1702581,
       2713058, 2525430, 2112638, 2447389, 2488269, 2399509,  336169,
       2800977, 2015063, 1117645,  604975,   52834, 2578235, 1971367,
        778874, 2700986, 2485703, 2092617, 2149247, 1493633, 2797098,
       2780575, 1573606, 1217277, 2796193, 2694909, 2245537, 1801526,
       2397118, 2798

In [281]:
from user_info import *

In [282]:
df = get_user_data_search_embed(uid_list)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 128/128 [00:00<00:00, 77886.39it/s]


In [283]:
df

,id,resume,keywords,current_designation,user_experience,professional_info,education_info
0,1838334,KUNAL CHOPRA | Email: kunalchopra26@gmail.com ...,Financial Reporting Analysis Finance Business_...,Financial Business Partner,3,"[{'id': 3123181, 'userId': None, 'designation'...","[{'id': 2109571, 'userId': 1838334, 'institute..."
1,1422653,"EXPERIENCE\nSPECIALIST, BETTER.COM, CHANDIGARH...",Financial Credit Mortgage Credit_Risk Operatio...,Specialist,4,"[{'id': 2033893, 'userId': None, 'designation'...","[{'id': 1611530, 'userId': 1422653, 'institute..."
2,2152366,Mudit Kumar Mahesh\nMale | maheshmudit@gmail.c...,Marketing Digital Financial Strategic Analytic...,Assistant Vice President - Corporate and Chie...,3,"[{'id': 3216342, 'userId': None, 'designation'...","[{'id': 2484906, 'userId': 2152366, 'institute..."
3,2786265,MANOJ REDDY\nMBA | Consulting| Strategy| Digit...,Digital Strategic Planning Strategy Digital_Tr...,Manager,9,"[{'id': 4216772, 'userId': None, 'designation'...","[{'id': 97700171, 'userId': 2786265, 'institut..."
4,1512430,"SACHIN KABRA\nMumbai, Maharashtra +91 80977810...",Financial Strategy Pricing Strategic TAX Inves...,General Manager - Strategy & Planning,6,"[{'id': 2177385, 'userId': None, 'designation'...","[{'id': 1719305, 'userId': 1512430, 'institute..."
...,...,...,...,...,...,...,...
120,1614881,Madhulika\nDegala\n+91 99081 95981\nmadhulika....,Marketing Strategy Lending Recruitment Analysi...,Strategy & Operations Manager,3,"[{'id': 3076333, 'userId': None, 'designation'...","[{'id': 1841482, 'userId': 1614881, 'institute..."
121,2403849,Profile\nProfessional experience of around 8 y...,Audit Financial Pivotal CFO Analytics Debtor D...,Assistant Manager,7,"[{'id': 3610742, 'userId': None, 'designation'...","[{'id': 94907595, 'userId': 2403849, 'institut..."
122,2549947,PRAGYA KEJRIWAL\nAge : 23 I Contact: +91 98300...,BI Power_BI Equity Valuation Proposal_Writing ...,Associate,3,"[{'id': 3903279, 'userId': None, 'designation'...","[{'id': 97472475, 'userId': 2549947, 'institut..."
123,2678664,SREEDHAR CHANDRAN\nM:+91 9880092146 E-Mail:\nc...,Finance Accounting Reconciliation MIS Statutor...,Deputy Finance Controller,9,"[{'id': 4056985, 'userId': None, 'designation'...","[{'id': 97585079, 'userId': 2678664, 'institut..."


In [284]:
# df[['id','resume']].sample(10).values

In [285]:
# df

In [286]:
# df['id']=df['id'].astype(int)

In [287]:
df1= df[['id','resume']]

In [288]:
df1

,id,resume
0,1838334,KUNAL CHOPRA | Email: kunalchopra26@gmail.com ...
1,1422653,"EXPERIENCE\nSPECIALIST, BETTER.COM, CHANDIGARH..."
2,2152366,Mudit Kumar Mahesh\nMale | maheshmudit@gmail.c...
3,2786265,MANOJ REDDY\nMBA | Consulting| Strategy| Digit...
4,1512430,"SACHIN KABRA\nMumbai, Maharashtra +91 80977810..."
...,...,...
120,1614881,Madhulika\nDegala\n+91 99081 95981\nmadhulika....
121,2403849,Profile\nProfessional experience of around 8 y...
122,2549947,PRAGYA KEJRIWAL\nAge : 23 I Contact: +91 98300...
123,2678664,SREEDHAR CHANDRAN\nM:+91 9880092146 E-Mail:\nc...


In [289]:
# df1 = df1.sample()

In [290]:
documents = []
for x in df1:
    y = 'id: '+df1['id'].values+' '+df1['resume'].values
    documents.append(y)

In [291]:
len(documents[0])

125

In [292]:
# documents[0]

In [293]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.vectorstores import FAISS


In [294]:
# documents = df1.values

In [295]:
# documents[90]

In [296]:
# documents[:, 0] = [{'id':int(value)} for value in documents[:, 0]]


In [297]:
# documents[0][90]

In [298]:
# len(documents)

In [299]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=1)
splits = text_splitter.split_text(str(documents[0]))
print(len(splits), splits[0])

1361 ['id: 1838334 KUNAL CHOPRA | Email: kunalchopra26@gmail.com | M: +91 9555789452 | LinkedIn\nACADEMIC QUALIFICATIONS\nDegree / Examination Institution Year\nPGDM Xavier Institute of Management &amp; Entrepreneurship 2021\nB.Com Sri Aurobindo College 2017\nClass XII MDH International School 2014\nClass X MDH International School 2012\nWORK EXPERIENCE\nCompany Description\nDECATHLON SPORTS\nINDIA\nTenure: May 2022 Present;\nRoles and Responsibilities: Finance Business Partner (E-Commerce)\n Owned


In [300]:
len(splits)

1361

In [301]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': 'cuda'})

In [302]:
db = FAISS.from_texts(documents[0], embeddings)


In [303]:
import langchain
from queue import Queue
from typing import Any
from langchain.llms.huggingface_text_gen_inference import HuggingFaceTextGenInference
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.schema import LLMResult
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts.prompt import PromptTemplate
from anyio.from_thread import start_blocking_portal #For model callback streaming
import re
import gradio as gr
langchain.debug=True 

In [304]:
model = 'meta-llama/Llama-2-7b-chat-hf'

In [157]:
# from langchain.llms import VLLM

# llm = VLLM(model=model,
#            trust_remote_code=True,  
#            max_new_tokens=128,
#            top_k=10,
#            top_p=0.95,
#            temperature=0.6,

# )

In [333]:
template = """
[INST]Use the following pieces of context to answer the question. If no context provided, answer like a AI assistant.
{context}
Question: {question} [/INST]
"""

retriever = db.as_retriever(
        search_kwargs={"k": 2}
    )

In [334]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=retriever,     
    chain_type_kwargs={
        "prompt": PromptTemplate(
            template=template,
            input_variables=["context", "question"],
        ),
    }
)

In [384]:
result = qa_chain({"query": "give me all users with their ids who have experience with UI/UX in resume text"})
# print(result)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "give me all users with their ids who have experience with UI/UX in resume text"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "give me all users with their ids who have experience with UI/UX in resume text",
  "context": "id: 2772792 Sanjeev Tarigopula\n+91-8088826168, sanjeev03bl@iimklive.com\nPhone: +91 8088826168 Email ID: sanjeev03bl@iimklive.com Banglore, India\nSummary:\n Leadership professional with ~4 years of experience across entrepreneurship, product management, strategy, tech &amp; innovation.\n Experience across tech innovation, web application building, product design &amp; stakeholder management.\n Stellar track record (Top 15 percentile in MBA &amp; Gold Medal Equivalent in Engineering) along with in

Processed prompts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.97s/it]

[llm/end] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain > 5:llm:VLLM] [4.98s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Sure, here are the users with their IDs who have experience with UI/UX in their resume text:\n\n1. Sanjeev Tarigopula - UI/UX experience mentioned in the work experience section under the role of \"Product Manager\" at Jio.\n2. Ayyappa Darla - UI/UX experience mentioned in the work experience section under the role of \"Senior at EY GDS India\" in the domain of Financial Reporting and operations under IFRS and US GAAP.\n\nPlease note that these are the users who have explicitly mentioned their",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] [4.98s] Exiting Chain run with output:
{
  "text": "Sure, here are the users with their IDs who have experience wi

In [385]:
result['result']

'Sure, here are the users with their IDs who have experience with UI/UX in their resume text:\n\n1. Sanjeev Tarigopula - UI/UX experience mentioned in the work experience section under the role of "Product Manager" at Jio.\n2. Ayyappa Darla - UI/UX experience mentioned in the work experience section under the role of "Senior at EY GDS India" in the domain of Financial Reporting and operations under IFRS and US GAAP.\n\nPlease note that these are the users who have explicitly mentioned their'

In [386]:
user_id_pattern = r'id: (\d+)' and r'ID: (\d+)' and r'ID is : (\d+)' and r"His ID is (\d+)" and r'ID: (\d+)' and r'Id: (\d+)' and r' ID: (\d+)'and r' id: (\d+)'and r'ID: (\d+)' and r' id: (\d+)'
# pattern = r"His ID is (\d+)"
# matches = re.findall(pattern, text)


user_ids = re.findall(user_id_pattern, result['result'])

In [380]:
user_ids

['2772792', '2795573']

In [381]:
data = get_user_data_search_embed(user_ids)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 34379.54it/s]


In [382]:
data

,id,resume,keywords,current_designation,user_experience,professional_info,education_info
0,2772792,"Sanjeev Tarigopula\n+91-8088826168, sanjeev03b...",Client Strategy Product_Management LEAN Busine...,Product Manager,4,"[{'id': 4194428, 'userId': None, 'designation'...","[{'id': 97684750, 'userId': 2772792, 'institut..."
1,2795573,"VAISHNAVI RAMESH KUMAR\nBangalore, India +91 9...",Audit Financial Chartered_Accountants Client C...,Manager,6,"[{'id': 4231943, 'userId': None, 'designation'...","[{'id': 97710895, 'userId': 2795573, 'institut..."


In [383]:
data[['keywords']].values

array([['Client Strategy Product_Management LEAN Business_Processes UI/UX Analytics Automation Marketing Trading Six_Sigma_Black_Belt Six_Sigma Black_Belt Data_Analytics Financial Venture_Capital Startup Digital Product_Roadmap Sales Consultant Institution HOD Sustainability IOT Production Strategy_Consultant Developer Recruitment Real_Estate Product_Design Lean_Six_Sigma Profitability MVP Research Engineering NPS Supply_Chain Product_Manager International_Research Analysis Stock Retention Leadership Stock_Market Prototype Corporate Project_Management'],
       ['Audit Financial Chartered_Accountants Client Communication Finance Networking Chartered_Accountant Textiles Strategy Monitoring Consultant Web_Services Operations Regulatory_Requirements Research IPCC Manufacturing Productivity Strategic Distribution Investing Analysis Accountant Leadership Senior_Consultant Financial_Control Financial_Services MS_Office Knowledge_Transfer Social_Networking Corporate TAX']],
      dtype=object

In [347]:
# data['resume'].values

In [315]:
# user_id_pattern